## 2.4. <a id='toc2_4_'></a>[Motion Control DIANA](#toc0_)

In motion control, we decided to only set speeds. The speeds are chosen with respect to the distance between the current position and the next point on the path and the angle between the current angle and the angle we’d like to have. The Kálmán filter and the global navigation give these values. The motion is a continuous exchange between turning and advancing phases.

3.3 Idea

This module of our project is used to give the wheels the necessary speeds to make the right movements. Three functions are present: the PID controller *PID_control()*, the turning function *turn()*, and the forward motion function *go_to_next_point()*. We switch between two phases while going from one point to the next. First, the turning phase where we use *turn()* to get in the angle that allows the robot to only move forward to reach the next point. Second, the advancing phase, which advences to the next point. The module is only called when there is no obstacle detected by the robot's proximity sensors. The functions work in a point-to-point and moment-to-moment fashion. This means that the angles taken into account are the current angle given by the Kálmán filter as well as the goal angle calculated between the first and second points of the path. The speeds are calculated for the distance between the two aforementioned points rather than from the current position to the final goal position.

3.3.1 PID Controller

The code has been inspired by the PI controller implemented in the course MICRO-315 given by Prof. Mondada. The PID controller is used for angle correction and is an integral part of both the turning and the forward motion phases. 

**Add a table like in Emilie's part**

Indeed, we look at the actual angle, *current_angle*, given by the filter,  and the goal angle of the robot, *robot.goal_angle* and using the difference between the two, *error, the integral error, *robot.int_error*, the previous error,*robot.prev_error*, as well as the time difference between two calls of the PID controller to calculate an optimal speed for attaining the desired angle. 

**Add some explenation about how get the time difference**

While in *turn()* it is the only calculation of speed, in *go_to_next_point()* it adds a rotational speed to a forward speed already calculated. The coefficients KP, KI and KD have been experimentally chosen during testing.

3.3.3 Forward Motion Function

Once the turning phase is done, as signalled by *goal_reached_t*, the advancing phase can be accessed. This function calculates and sends speeds similar to the turning phase. The speed is proportionally controlled to slow down and smoothly reach the next point. The coefficient K has also been experimentally chosen during testing. We use the PID controller to add a rotational speed in order to correct the angle and so to make sure that the robot stays on track. <br>
Once the goal is reached, the second point in the path is discarded (the first one in the current position as given by the camera) and a new goal angle is calculated using the *setAngle()* function found in the file *classes.py* . A helper variable goal_reached_f is used to signal that the motion phase has ended and a new turning motion can begin.

In [ ]:
"Add code to show how this this works between two points"
from tdmclient import ClientAsync
import motion_control
from classes import Thymio

client = ClientAsync()
node = await client.wait_for_node()
await node.lock()
await node.wait_for_variables()

Thym=Thymio()
Thym.path=[[0,10],[2,50]]
Thym.pos_X=Thym.path[0][0]
Thym.pos_Y=Thym.path[0][1]
Thym.setAngle()

In [ ]:
print(Thym.goal_angle)

3.3.4 Management of reaching the goal position

When the length of the path is down to one, the goal position has been reached and the work is done. We see this in the code for *go_to_next_point()*, where there is a condition that skips the function if the path length is too short. The speeds are set to 0 as a advancing phase has been completed, and no new angle is calculated nor is the path changed anymore, as only the current position is left. 

In [ ]:
def go_to_next_point(current_angle, current_position, obstacle, robot, node): 
    if len(robot.path) > 1:
        "code"

        else:                               #advancing phase in completed when close enough to the desired point
            robot.goal_reached_f = True
            robot.goal_reached_t = False
            robot.prev_error = 0
            robot.int_error = 0
            robot.prev_time = 0
            robot.setSpeedRight(0,node)
            robot.setSpeedLeft(0,node)
            robot.path.pop(1)               #path shortened
            if len(robot.path) > 1:
                robot.setAngle()            #angle changed with respect to the new goal
                
    else:
        pass                                # function skipped

As seen in the code above, the condition has to be introduced in the advancing phase completion as the *robot.setAngle()* function takes the shortened path as an input before it is taken into account by the whole function in a future call.